# Example of handling bad molecules

If you give a bad conformer, of course xTB will not be able to calculate any properties.
This is an example of how ppqm handles that


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import logging
import sys

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem.Draw import MolsToGridImage

In [ ]:
try:
    import ppqm
except ModuleNotFoundError:
    import pathlib

    cwd = pathlib.Path().resolve().parent
    sys.path.append(str(cwd))
    import ppqm

In [ ]:
from ppqm import jupyter as ppqm_jupyter

## Set logging level

In [ ]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger("ppqm").setLevel(logging.INFO)
logging.getLogger("xtb").setLevel(logging.DEBUG)
show_progress = False

## Define a molecule you like

In [ ]:
smiles = "Cc1cc(NCCO)nc(-c2ccc(Br)cc2)n1"  # CHEMBL1956589
molobj = Chem.MolFromSmiles(smiles)

In [ ]:
molobj

## Get a conformer, and make it physically impossible

- Get a conformer from rdkit
- Change bond length to make it bad


In [ ]:
molobj = ppqm.tasks.generate_conformers(molobj, n_conformers=1)

In [ ]:
molobj

In [ ]:
coordinates = ppqm.chembridge.get_coordinates(molobj)

In [ ]:
atoms = ppqm.chembridge.get_atoms(molobj)

In [ ]:
atoms

In [ ]:
coordinates[0, :] = 0.0
coordinates[1, :] = 0.0

In [ ]:
ppqm.chembridge.molobj_set_coordinates(molobj, coordinates)

In [ ]:
ppqm_jupyter.show_molobj(molobj)

## Try to optimize this bad conformer

In [ ]:
xtb = ppqm.XtbCalculator(scr="_tmp_directory_", n_cores=1, cmd="xtb")

In [ ]:
calculation = {
    "gfn": 2,
    "alpb": "water",
    "opt": None,
}

In [ ]:
results = xtb.calculate(molobj, calculation)

In [ ]:
results

And we can see that we see the xTB error message in the log, and the returning result is None